In [23]:
import pandas as pd

from scipy import stats

import numpy as np

import seaborn as sns

from scipy.stats import chi2_contingency
df = pd.read_csv('House_price.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area               665 non-null    float64
 1   price              617 non-null    float64
 2   postDate           671 non-null    int64  
 3   land_certificate   369 non-null    object 
 4   house_direction    42 non-null     object 
 5   balcony_direction  24 non-null     object 
 6   toilet             240 non-null    float64
 7   bedroom            320 non-null    float64
 8   floor              366 non-null    float64
 9   lat                671 non-null    float64
 10  long               671 non-null    float64
 11  property_type      671 non-null    object 
dtypes: float64(7), int64(1), object(4)
memory usage: 63.0+ KB


In [24]:
df['land_certificate'].unique()

array(['So do', nan], dtype=object)

1. Giữa giá nhà và diện tích có liên quan với nhau không?

In [7]:
df1 = df.dropna(subset=['area','price'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 670
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area               615 non-null    float64
 1   price              615 non-null    float64
 2   postDate           615 non-null    int64  
 3   land_certificate   359 non-null    object 
 4   house_direction    41 non-null     object 
 5   balcony_direction  24 non-null     object 
 6   toilet             237 non-null    float64
 7   bedroom            316 non-null    float64
 8   floor              357 non-null    float64
 9   lat                615 non-null    float64
 10  long               615 non-null    float64
 11  property_type      615 non-null    object 
dtypes: float64(7), int64(1), object(4)
memory usage: 62.5+ KB


Phát biểu giả thuyết:
+ H0: Giữa giá nhà và diện tích không có tương quan với nhau
+ H1: Giữa giá nhà và diện tích có mối tương quan với nhau

In [9]:
r, p_value = stats.pearsonr(df1.area, df1.price)
print(f'r: {r}; p_value: {p_value}')

r: 0.22852101906798145; p_value: 9.947613326077062e-09


Kết luận: chấp nhận giả thuyết H1


--------------------------------------------------------------------
2. Kiểm định tương quan giá nhà và tọa độ địa lý


In [12]:
df2 = df.dropna()

Phát biểu giả thuyết:
+ H0: Giữa giá nhà và tọa độ địa lý không có mối tương quan với nhau
+ H1: Giữa nhà và tọa độ địa lý có mối tương quan với nhau

In [14]:
r, p_value = stats.pearsonr(df2.long, df2.price)
print(f'r: {r}; p_value: {p_value}')

r: 0.26783280265426535; p_value: 0.35456112847319127


Kết luận: Do P > 0.05 vì vậy chấp nhận giả thuyết H0

3. Thuộc tính land_certificate và property_type có tương quan với nhau?

In [39]:
df3 = df.filter(['land_certificate','property_type'])
df3.head()

df3 = df3.fillna('Khong so do')
df3

,land_certificate,property_type
0,So do,trong ngo
1,So do,trong ngo
2,So do,trong ngo
3,So do,trong ngo
4,So do,trong ngo
...,...,...
666,Khong so do,mat pho
667,So do,mat pho
668,Khong so do,mat pho
669,Khong so do,mat pho


Phát biểu giả thuyết:
+ H0: Hai thuộc tính land_certificate và property_type không có mối liên hệ
+ H1: Hai thuộc tính land_certificate và property_type có mối liên hệ

In [40]:
contigency= pd.crosstab(df3['land_certificate'], df3['property_type'])
contigency

property_type,mat pho,trong ngo
land_certificate,,
Khong so do,108,194
So do,114,255


In [42]:
c, p, dof, expected = chi2_contingency(contigency)

print(p)

0.21106331497658834


Kết luận: Do P >0.05, chấp nhận giả thuyết H0

4. Mối tương quan giá đơn vị- ordinal với tọa độ

In [54]:
df4 = df.dropna(subset=['area','price'])

df4['Unit_price']=df['price']/df['area'] 

q1 = df4.Unit_price.quantile(0.25)

q2 = df4.Unit_price.quantile(0.5)

q3 =df4.Unit_price.quantile(0.75)


condition = [df4.Unit_price < q1, df4.Unit_price<q2,df4.Unit_price < q3]

values = [1,2,3]

df4['Unitprice_ordinal'] = np.select(condition, values,default=4)


df4.head()

/tmp/ipykernel_38223/3922554829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Unit_price']=df['price']/df['area']
/tmp/ipykernel_38223/3922554829.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Unitprice_ordinal'] = np.select(condition, values,default=4)


,area,price,postDate,land_certificate,house_direction,balcony_direction,toilet,bedroom,floor,lat,long,property_type,Unit_price,Unitprice_ordinal
0,50.0,14700.0,44197,So do,NaN,NaN,4.0,6.0,6.0,21.015698,105.824583,trong ngo,294.000000,4
1,57.0,4200.0,44265,So do,Nam,Nam,3.0,8.0,4.0,21.025208,105.833111,trong ngo,73.684211,1
2,62.0,11000.0,44268,So do,NaN,dong-Nam,6.0,5.0,6.0,21.025379,105.801011,trong ngo,177.419355,3
3,26.0,2900.0,44292,So do,NaN,NaN,4.0,3.0,4.0,21.014325,105.834918,trong ngo,111.538462,2
4,40.0,3950.0,44292,So do,NaN,NaN,NaN,NaN,5.0,21.009862,105.838083,trong ngo,98.750000,1


Phát biếu giả thuyết:
+ H0: Giữa unit_price_ordinal và lat không có mối tương quan 
+ H1: Giữa unit_price_ordinal và lat có mối tương quan

In [56]:
r, p_value =stats.spearmanr(df4.lat,df4.Unitprice_ordinal)

print(f'r: {r}; p_value = {p_value}')


r: 0.016726353923742526; p_value = 0.6788849673606929


Kết luận: Với P >0.05 vì vậy chấp nhận giả thuyết H0